<a href="https://colab.research.google.com/github/Tahabpoker/making_gpt_for_fun/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# http://youtube.com/watch?v=kCc8FmEb1nY
# time = 33:57# wow

In [39]:
with open('input.txt', 'r') as f:
    text = f.read()

In [40]:
len(text)

1115394

In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [43]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [44]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [45]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hello there"))
print(decode([2,2,2,3,4,5,45,40,32,3,1,1]))


[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
!!!$&'gbT$  


In [46]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [47]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [48]:
print(val_data[:100])


tensor([12,  0,  0, 19, 30, 17, 25, 21, 27, 10,  0, 19, 53, 53, 42,  1, 51, 53,
        56, 56, 53, 61,  6,  1, 52, 43, 47, 45, 46, 40, 53, 59, 56,  1, 14, 39,
        54, 58, 47, 57, 58, 39,  8,  0,  0, 14, 13, 28, 32, 21, 31, 32, 13, 10,
         0, 19, 53, 53, 42,  1, 51, 53, 56, 56, 53, 61,  6,  1, 52, 43, 47, 45,
        46, 40, 53, 59, 56,  1, 19, 56, 43, 51, 47, 53,  8,  0, 19, 53, 42,  1,
        57, 39, 60, 43,  1, 63, 53, 59,  6,  1])


In [49]:
decode(train_data[:100].tolist())

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [50]:
decode(val_data[:100].tolist())

'?\n\nGREMIO:\nGood morrow, neighbour Baptista.\n\nBAPTISTA:\nGood morrow, neighbour Gremio.\nGod save you, '

In [51]:
block_size = 8


In [52]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"{context} , = {target}")

tensor([18]) , = 47
tensor([18, 47]) , = 56
tensor([18, 47, 56]) , = 57
tensor([18, 47, 56, 57]) , = 58
tensor([18, 47, 56, 57, 58]) , = 1
tensor([18, 47, 56, 57, 58,  1]) , = 15
tensor([18, 47, 56, 57, 58,  1, 15]) , = 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) , = 58


In [53]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device=device), y.to(device=device)
  return x, y

xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)



torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')


In [54]:
eval_iters = 200

In [61]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train","val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [57]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__ (self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb,yb)
# print(loss)
# print(logits)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))



pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [58]:
optimizer = torch.optim.AdamW(m.parameters(), lr =1e-3)

In [66]:
batch_size = 32
max_iters = 1000
eval_intervals = 300
learning_rate = 1e-3

for iter in range(max_iters):
  if iter % eval_intervals ==  0:
    losses = estimate_loss()
    print(f"step {iter} train_loss: {losses["train"]}, val_loss: {losses["val"]}")
  xb, yb = get_batch("train")

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)



step 0 train_loss: 2.4583675861358643, val_loss: 2.4807333946228027
step 300 train_loss: 2.454728364944458, val_loss: 2.48295521736145
step 600 train_loss: 2.447457790374756, val_loss: 2.4888827800750732
step 900 train_loss: 2.4488697052001953, val_loss: 2.481710910797119


In [67]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))



ALARI menatoulinor wosing brwimapisexQUSwe e wo IICis;
ST:
GRDo y'silll whertaperulllalop!
Antheolirconourjura d f t wothr u th?
Whaling mas, h n t il c; d junlotir Whth'dlll my ay meras be:
HERGSo ndonbust RIUCE:


Anveaban; IOn hequ thitan nu tonm TESCorte y, d.
Sl!
HAUSTRoy IE:
ARERDO:
BRolers e
Thins; s ookesthouk bl,-der, s, es s;
RICld t olk ve, scee Clll!
Y theeee wheresed we h, st ar
ENCELONTherveirake mes ht knt
NG thin.


'd!
Wheseson rwshe. ar?
EEY: s IA:
Whthathe couf me, sindoass oo
